# Python 機械学習

Python による機械学習を学ぶためのノートブック

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RyoWakabayashi/python-learning/blob/main/notebooks/Python%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92.ipynb)

## 画像識別AIの仕組み

人間の脳（ニューラルネットワーク）を模して、画像を認識する

各画像を数値の組み合わせ（行列）としてニューラルネットワークに入力し、分類していく

https://playground.tensorflow.org/

ニューラルネットワークの構造、パラメータの組み合わせをモデルという

## Google Colab のランタイム

ランタイムのタイプを GPU に変更する

Colab Pro を契約すると、より強力な GPU が選択可能になる

https://colab.research.google.com/signup/pricing?hl=ja

## TensorFlow による推論

TensorFlow は機械学習フレームワークの1種

https://www.tensorflow.org/?hl=ja

TensorFlow Hub に様々な AI モデルが公開されている

https://tfhub.dev/

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

画像識別モデルを TensorFlow Hub から読み込む

In [ ]:
MODULE_HANDLE = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"
pixels = 299
IMAGE_SIZE = (pixels, pixels)

classifier = hub.load(MODULE_HANDLE)

FiftyOne を使ってテスト用画像を取得する

https://docs.voxel51.com/

様々なデータセット（データの集まり）からデータを簡単にダウンロードできる

In [ ]:
!pip install fiftyone

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

Google の Open Images Dataset V6 から猫と犬の画像を取得する

In [ ]:
dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["detections"],
    classes=["Cat", "Dog"],
    max_samples=10,
    only_matching=True,
)

ノートブック上で画像、アノテーションデータ（分類情報、位置情報など、付加されたデータ）を確認することができる

In [ ]:
session = fo.launch_app(dataset)

データセットを画像識別用の形式でエクスポートする

In [ ]:
dataset.export(
    export_dir="./classification",
    dataset_type=fo.types.ImageClassificationDirectoryTree,
)

In [ ]:
import cv2
import numpy as np
from IPython.display import display, Image

In [ ]:
def show(img):
    _, buf = cv2.imencode(".jpg", img)
    display(Image(data=buf.tobytes()))

In [ ]:
image = cv2.imread("classification/Cat/000001.jpg")

show(image)

画像を推論のための形式に変換する

In [ ]:
image

In [ ]:
# 0 から 255 で表される色の範囲を 0 から 1 に変換
input = cv2.resize(image, IMAGE_SIZE) / 255.

input

In [ ]:
input.shape

In [ ]:
# 先頭に次元を追加
input = input[np.newaxis,:,:,:]

input.shape

In [ ]:
# 型を float32 のテンソルに変換
input = tf.image.convert_image_dtype(input, tf.float32)

input

推論を実行する

In [ ]:
# 推論
results = classifier(input)

results

In [ ]:
# Softmax 関数によって、推論結果をクラス毎の確率に変換
probabilities = tf.nn.softmax(results).numpy()

probabilities

In [ ]:
# 確率の高い順に並び替えて、上位5つを取得
top_5 = tf.argsort(probabilities, axis=-1, direction="DESCENDING")[0][:5].numpy()

top_5

識別対象は ImageNet のクラス (1,000 種類)

https://www.image-net.org/

In [ ]:
labels_file = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"

downloaded_file = tf.keras.utils.get_file("labels.txt", origin=labels_file)

classes = []

with open(downloaded_file) as f:
  labels = f.readlines()
  classes = [l.strip() for l in labels]

classes

In [ ]:
# 確率の高い順にクラス名と確率を表示
for class_index in top_5:
    class_label = classes[class_index]
    line = f'{round(probabilities[0][class_index] * 100, 1)} % {class_label}'
    print(line)

推論結果のラベルを日本語化するために deep_translator をインストールする

In [ ]:
!pip install deep_translator

In [ ]:
from deep_translator import GoogleTranslator

In [ ]:
# 確率の高い順にクラス名と確率を表示
for class_index in top_5:
    class_label = GoogleTranslator(source='auto', target='ja').translate(classes[class_index])
    line = f'{round(probabilities[0][class_index] * 100, 1)} % {class_label}'
    print(line)

一連の推論処理を関数化する

In [ ]:
def classify(image_classifier, image_path, class_list):
    print(image_path)
    image = cv2.imread(image_path)

    show(image)

    resized_image = cv2.resize(image, IMAGE_SIZE) / 255.
    input = tf.image.convert_image_dtype(resized_image[np.newaxis,:,:,:], tf.float32)

    probabilities = tf.nn.softmax(image_classifier(input)).numpy()

    top_5 = tf.argsort(probabilities, axis=-1, direction="DESCENDING")[0][:5].numpy()

    for class_index in top_5:
        class_label = GoogleTranslator(source='auto', target='ja').translate(class_list[class_index])
        line = f'{round(probabilities[0][class_index] * 100, 1)} % {class_label}'
        print(line)

    return class_list[top_5[0]]

In [ ]:
classify(classifier, "classification/Dog/000003.jpg", classes)

## TensorFlow による機械学習トレーニング

### 転移学習（Transfer Learning）

学習済みモデルをベースにトレーニングする

ニューラルネットワークの構造は同じで、最後の識別対象だけを変更し、再トレーニングする

0から学習よりも少ないデータで早く学習できる

In [ ]:
MODULE_HANDLE = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5"

自分で識別したいクラス（トレーニング対象）を Open Images Dataset から選択する

https://storage.googleapis.com/openimages/web/index.html

In [ ]:
# 自分の選択したクラスに変更する
my_classes = sorted(["Turtle", "Rabbit"])

my_classes

In [ ]:
print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(
        input_shape=IMAGE_SIZE + (3,)   # 入力層の形は画像の 横 * 縦 * 3（RGBの色）
    ),
    hub.KerasLayer(
        MODULE_HANDLE,                  # 元モデル
        trainable=False                 # 元モデルの中間層は学習しない
    ),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(
        len(my_classes),                # 出力層の形は識別する種類の数
        kernel_regularizer=tf.keras.regularizers.l2(0.0001)
    )
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

model.compile(
    optimizer=tf.keras.optimizers.SGD(
        learning_rate=0.00002,          #学習率
        momentum=0.9
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=True,
        label_smoothing=0.1
    ),
    metrics=['accuracy']
)

### トレーニングデータの準備

In [ ]:
# ダウンロード
my_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="test",
    label_types=["detections"],
    classes=my_classes,
    max_samples=250,
)

In [ ]:
session = fo.launch_app(my_dataset)

In [ ]:
# エクスポート
data_dir = "my_dataset"

my_dataset.export(
    export_dir=data_dir,
    dataset_type=fo.types.ImageClassificationDirectoryTree,
    classes=my_classes,
)

In [ ]:
import glob
import shutil

識別対象の画像以外は削除する

In [ ]:
directories = glob.glob(data_dir + '/*')

for directory in directories:
    if not directory.split("/")[-1] in my_classes:
        shutil.rmtree(directory)

トレーニング用データ、検証用データをそれぞれ準備する

- トレーニング用データ（トレーニング時に読み込むするデータ）
- 検証用データ（トレーニング中に精度を確認するためのデータ）

トレーニングの流れ

- epoch を繰り返す
  - トレーニング用データでパラメータを調整する
  - 検証用データで精度を確認する
- 一番いい状態を保存する

In [ ]:
# データ生成の共通処理
datagen_kwargs = dict(
    rescale=1./255,       # RGB を 1 ～ 255 の整数で表していたのを 0.0 ～ 1.0 の小数で表すようにする
    validation_split=.20  # データ全体のうち、2割は検証データにする
)

dataflow_kwargs = dict(
    target_size=IMAGE_SIZE,  # 画像サイズ
    batch_size=8,            # 並列実行数
    interpolation="bilinear" # リサイズの方法
)

In [ ]:
# 検証用データの生成定義
# 検証用データは加工しない
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs
)

valid_generator = valid_datagen.flow_from_directory(
    data_dir,
    subset="validation",
    shuffle=False,
    **dataflow_kwargs
)

### トレーニング用データの水増し

同じ画像ばかり学習すると、過学習（偏った学習により、未知のデータに弱くなる現象）が発生してしまう

画像加工をランダムに加えることで、画像のバリエーションを増やす

In [ ]:
import random
from tensorflow.keras.preprocessing import image

In [ ]:
files = glob.glob(data_dir + '/*/*.jpg') #　学習画像一覧を取得

# 一覧からランダムに一つ選択しPIL形式でオープン
img = image.load_img(random.choice(files))
# PIL形式をnumpyのndarray形式に変換
x_img = image.img_to_array(img)
x_img = cv2.resize(x_img, IMAGE_SIZE)

show(cv2.cvtColor(x_img, cv2.COLOR_RGB2BGR))

# (height, width, 3) -> (1, height, width, 3)
x_img = x_img.reshape((1,) + x_img.shape)

In [ ]:
def check_datagen(train_datagen):
    max_img_num = 4
    images = []
    for data in train_datagen.flow(x_img, batch_size=1):
        images.append(data[0])
        # datagen.flowは無限ループするため必要な枚数取得できたらループを抜ける
        if (len(images) % max_img_num) == 0:
            break

    for img in images:
        show(cv2.cvtColor(img * 255, cv2.COLOR_RGB2BGR))

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=45,  # 回転範囲 整数を指定 rotation_range=45の場合、-45°~45°の間でランダムに回転する
    **datagen_kwargs
)

check_datagen(train_datagen)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,  # 左右反転 True,Falseを指定 Trueの場合ランダムに左右反転する
    vertical_flip=True,    # 上下反転 True,Falseを指定 Trueの場合ランダムに上下反転する
    **datagen_kwargs
)

check_datagen(train_datagen)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.3,   # 左右並行移動 範囲[0, 1]の小数を指定する width_shift_range=0.3の場合、[-0.3 * 幅, 0.3 * 幅] の範囲でランダムに左右平行移動する。
    height_shift_range=0.3,  # 上下並行移動 範囲[0, 1]の小数を指定する height_shift_range=0.3の場合、[-0.3 * 高さ, 0.3 * 高さ] の範囲でランダムに上下平行移動する。
    **datagen_kwargs
)

check_datagen(train_datagen)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    channel_shift_range=50,  # 輝度変化 範囲[0, 255]の値を指定する channel_shift_range=50 とした場合、[-50, 50] の範囲でランダムに画素値に値を足す。
    **datagen_kwargs
)

check_datagen(train_datagen)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    shear_range=50,         # 斜め方向の歪み 少数または整数を指定 shear_range=50の場合、-50° ~ 50° の範囲でランダムに歪みを加える。
    **datagen_kwargs
)

check_datagen(train_datagen)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    zoom_range=0.3,          # 拡大・縮小 範囲[0, 1]の小数を指定する zoom_range=0.3の場合、[1 - 0.3, 1 + 0.3] つまり [0.7, 1.3] の範囲でランダムに拡大縮小する。
    **datagen_kwargs
)

check_datagen(train_datagen)

In [ ]:
# 色調調整をする関数
def color_shift(image):
    image = np.array(image)   
    shift_range = random.uniform(100, -100) #各画素値に足す値の範囲を[0, 255]で指定 数が大きければより原色に近くなる
    rgb = random.randint(0, 2)
    # RGBそれぞれに対しランダムに値を足す
    image[:, :, rgb] += shift_range

    return image

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = color_shift, # 自前で用意した前処理関数を指定する
    **datagen_kwargs
)

check_datagen(train_datagen)

In [ ]:
# 色調調整をする関数
def color_shift(image):
    image = np.array(image)   
    shift_range = random.uniform(100, -100) #各画素値に足す値の範囲を[0, 255]で指定 数が大きければより原色に近くなる
    rgb = random.randint(0, 2)
    # RGBそれぞれに対しランダムに値を足す
    image[:, :, rgb] += shift_range

    return image

# 学習データの生成定義
do_data_augmentation = True
if do_data_augmentation:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=45,       # 回転範囲 整数を指定
        horizontal_flip=True,      # 左右反転 True,Falseを指定
        vertical_flip=True,        # 上下反転　True,Falseを指定
        width_shift_range=0.3,   # 左右並行移動 範囲[0, 1]の小数を指定する
        height_shift_range=0.3,  # 上下並行移動 範囲[0, 1]の小数を指定する
        channel_shift_range=100,  # 輝度変化 範囲[0, 255]の値を指定する
        shear_range=50,           # 斜め方向の歪み 少数または整数を指定
        zoom_range=0.5,          # 拡大・縮小
        preprocessing_function = color_shift, # 自前で用意した前処理関数を指定　例では色調調整をする関数を指定
        **datagen_kwargs
    )
else:
    train_datagen = valid_datagen

check_datagen(train_datagen)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    data_dir, 
    subset="training", 
    shuffle=True, 
    **dataflow_kwargs
)

### 早期終了（アーリーストッピング）

過学習が発生すると、学習すればするほど検証データに対する精度が落ちる

一定回数か学習の兆候が見えたら、その時点でトレーニングを終了する

In [ ]:
cb_es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,            # 5回連続で val_loss が高くなったとき、学習を終了する
    verbose=1,
    mode='auto'
)

### トレーニングの実行

In [ ]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size

hist = model.fit(
    train_generator,
    epochs=50,            #最大何epoch実行するか指定  各自で調整するが、大きくしすぎるとメモリが足りなくなるため200以上は指定しないこと
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps,
    callbacks=[cb_es]
).history

### トレーニング結果（モデル）の保存

In [ ]:
saved_model_path = "/content/saved_model"
tf.keras.models.save_model(model, saved_model_path)

# クラス一覧のテキストファイルをモデルファイルと同じディレクトリに作成する
with open(saved_model_path + '/labels.txt', 'w') as f:
    for category in my_classes:
        f.write("%s\n" % category)

## トレーニング結果の視覚化

In [ ]:
import matplotlib.pylab as plt

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])

## モデルの Google Drive への保存

Google Colab 上のデータは、セッションが切れると消えてしまう

残しておきたいものは必ず外部に保存する

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DIR_NAME = "20230516"

SAVED_MODEL_PATH = "/content/saved_model/"

DRIVE_SAVED_PATH = "/content/drive/MyDrive/saved_model/{}/".format(DIR_NAME)

# ドライブに保存
!mkdir -p $DRIVE_SAVED_PATH
!cp -pR "$SAVED_MODEL_PATH"* $DRIVE_SAVED_PATH

## トレーニングしたモデルによる推論

Google Drive からモデルを読み込む

In [ ]:
new_model = tf.keras.models.load_model(DRIVE_SAVED_PATH)
new_model.summary()

テスト用データの取得

In [ ]:
test_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["detections"],
    classes=my_classes,
    max_samples=100,
)

In [ ]:
test_dir = "test"

test_dataset.export(
    export_dir=test_dir,
    dataset_type=fo.types.ImageClassificationDirectoryTree,
    classes=my_classes,
)

In [ ]:
for directory in glob.glob(test_dir + '/*'):
    if not directory.split("/")[-1] in my_classes:
        shutil.rmtree(directory)

In [ ]:
for category in my_classes:
    # クラスごとに最初の一件のみ分類にかける
    image_path_list = glob.glob("{}/{}/*".format(test_dir, category))
    img_pth = image_path_list[1]
    classify(new_model, img_pth, my_classes)

In [ ]:
import os

In [ ]:
acc_results_dict = {}
y_pred = []
y_true = []
total = 0
total_t_data = 0

def predicted_class(image_path_list, category, image_num, size, new_model, y_true, y_pred):
    t_data = 0
    f_data = 0

    for img_pth in image_path_list:
        predicted_class_name = classify(new_model, img_pth, my_classes)

        if predicted_class_name == category:
          t_data = t_data + 1
        else:
          f_data = f_data + 1

        print(predicted_class_name == category, '\n')

        y_true.append(category)
        y_pred.append(predicted_class_name)
        presult = [t_data, f_data]
        accresult = t_data / image_num

    return presult, accresult, y_true, y_pred, t_data

for category in my_classes:
    # フォルダごとに識別にかける
    image_path_list = glob.glob("{}/{}/*".format(test_dir, category))
    num = len(os.listdir("{}/{}".format(test_dir, category)))
    total += num
    p_result, acc_result , y_true, y_pred, t_data = predicted_class(
        image_path_list = image_path_list, 
        category = category, 
        image_num = num, 
        size = IMAGE_SIZE, 
        new_model =new_model, 
        y_true = y_true, 
        y_pred = y_pred
    )

    acc_results_dict[category] = acc_result
    total_t_data += t_data
    print('-------------------------------------------')

print('accuracy')

# 正解率を表示
for class_name, accuracy in acc_results_dict.items() :
    print('{} : {}%'.format(class_name, accuracy * 100))

print('total : {}%'.format(total_t_data / total * 100))